# Quantum Protocol: Probabilistic Communication Channels

In this tutorial, we model a binary communication channel where bits can
be flipped during transmission. We use ProbFlow's **Bernoulli** distribution
to model bit errors, a **Beta** distribution as a prior on channel reliability,
and compose distributions to analyze end-to-end protocol reliability.

## Setup

In [ ]:
import numpy as np
from probflow import Bernoulli, Beta, ProbFlow

## Modeling a Noisy Channel

A binary symmetric channel flips each transmitted bit with some probability.
We model successful transmission (no flip) as a Bernoulli trial with
probability `p = 0.95`.

In [ ]:
channel = Bernoulli(p=0.95)
transmissions = channel.sample(20)
print(f"20 transmissions (1=success): {transmissions}")
print(f"Success rate: {np.mean(transmissions):.2f}")

## Probability Mass Function

The PMF gives us the probability of success (1) and failure (0).

In [ ]:
print(f"P(success) = {channel.pdf(1):.4f}")
print(f"P(failure) = {channel.pdf(0):.4f}")

## Beta Prior on Reliability

In practice, the true channel reliability `p` is unknown. We use a
Beta distribution as a prior. A `Beta(alpha=20, beta=2)` encodes our
belief that the channel is reliable but with some uncertainty.

In [ ]:
reliability_prior = Beta(alpha=20, beta=2)
prior_samples = reliability_prior.sample(1000)
print(f"Prior mean reliability: {np.mean(prior_samples):.4f}")
print(f"Prior std: {np.std(prior_samples):.4f}")

## Evaluating the Prior

We can evaluate how likely specific reliability values are under our prior.

In [ ]:
test_values = np.array([0.80, 0.85, 0.90, 0.95, 0.99])
densities = reliability_prior.pdf(test_values)
for v, d in zip(test_values, densities):
    print(f"  PDF at p={v:.2f}: {d:.4f}")

## Probability the Channel is Highly Reliable

What is the probability that the channel reliability exceeds 0.90?

In [ ]:
prob_above_90 = 1.0 - reliability_prior.cdf(0.90)
print(f"P(reliability > 0.90) = {prob_above_90:.4f}")

## Two-Hop Protocol

A message passes through two independent channels in series.
The end-to-end success requires **both** hops to succeed.
We model this using the product (`*`) of two Bernoulli distributions.

In [ ]:
hop1 = Bernoulli(p=0.95)
hop2 = Bernoulli(p=0.90)
end_to_end = hop1 * hop2

In [ ]:
e2e_samples = end_to_end.sample(10000)
print(f"End-to-end success rate: {np.mean(e2e_samples):.4f}")
print(f"Expected (0.95 * 0.90): {0.95 * 0.90:.4f}")

## Joint Distribution of Channel Reliabilities

Using the `&` operator, we can model the joint distribution of
two independent channel reliability priors.

In [ ]:
prior1 = Beta(alpha=20, beta=2)
prior2 = Beta(alpha=15, beta=3)
joint_prior = prior1 & prior2

In [ ]:
joint_samples = joint_prior.sample(1000)
print(f"Joint samples shape: {joint_samples.shape}")
print(f"Channel 1 mean: {np.mean(joint_samples[:, 0]):.4f}")
print(f"Channel 2 mean: {np.mean(joint_samples[:, 1]):.4f}")

## Protocol Model with ProbFlow

We define the complete protocol model using the ProbFlow context manager.

In [ ]:
with ProbFlow() as protocol_model:
    ch_prior = Beta(alpha=20, beta=2)
    protocol_model.add_distribution(ch_prior, name="channel_reliability")

    channel_obs = Bernoulli(p=0.95)
    protocol_model.add_distribution(channel_obs, name="observed_transmissions")

print(f"Model distributions: {list(protocol_model.variables.keys())}")

## Credible Interval for Reliability

Using the quantile function on the Beta prior, we can find a
90% credible interval for channel reliability.

In [ ]:
retrieved = protocol_model.get_distribution("channel_reliability")
lower = retrieved.quantile(0.05)
upper = retrieved.quantile(0.95)
print(f"90% credible interval: [{lower:.4f}, {upper:.4f}]")

## Summary

In this tutorial we:
- Modeled bit errors with **Bernoulli** distributions
- Used a **Beta** prior to represent uncertainty in channel reliability
- Composed channels with `*` for multi-hop protocol reliability
- Created **joint distributions** with `&` for multiple channels
- Built a full model with the **ProbFlow** context manager
- Computed credible intervals using the **quantile** function